In [1]:
import rebound
import reboundx
import numpy as np
import sys
import os
import re
import pandas as pd

df = pd.DataFrame(columns=('filename', 'tinstability'))
datapath = 'data/'
for filename in os.listdir(datapath):
    result = re.search(r"nonresIC(.*).bin", filename)
    if result:
        simID = int(result.group(1))
        df.loc[simID] = [filename, np.nan]
        
df = df.sort_index()
df.tail()

,filename,tinstability
195,nonresIC195.bin,NaN
196,nonresIC196.bin,NaN
197,nonresIC197.bin,NaN
198,nonresIC198.bin,NaN
199,nonresIC199.bin,NaN


In [2]:
def instability_time(row):
    sim = rebound.Simulation.from_archive(datapath+row['filename'])
    row['tinstability'] = sim.t
    return row

In [3]:
%%time
df = df.apply(instability_time, axis=1)

/mnt/raid-cita/dtamayo/p3new/lib/python3.4/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 107 ms, sys: 300 ms, total: 407 ms
Wall time: 1.34 s


We bin all simulations that lasted less than 1000 years into a single instability time bin at 1000:

In [4]:
def setmin(row):
    if row['tinstability'] == 0:
        row['tinstability'] = 1000
    return row

df = df.apply(setmin, axis=1)
df.head()

,filename,tinstability
0,nonresIC0.bin,1.000000e+03
1,nonresIC1.bin,2.000000e+03
2,nonresIC2.bin,5.000000e+06
3,nonresIC3.bin,1.000000e+03
4,nonresIC4.bin,5.000000e+03


In [5]:
df.to_csv('csvs/nonresinstabilitytimes.csv', encoding='ascii')